In [1]:
version = "REPLACE_PACKAGE_VERSION"

---
# Assignment 2 Part 2: Query Log Analysis (50 pts)



If you aspired to run a successful information retrieval system like a search engine, you would definitely like to know your users better so that you can better serve them (and spam them with more targeted ads). One important way of doing so is analysing what they have been searching for, namely, a query log. 

In this assignment we will analyse [a query log data set](https://github.com/microsoft/BingCoronavirusQuerySet) curated by Microsoft Bing Search. Data from April 2020 to March 2021 (inclusive) is provided under `assets/BingCoronavirusQuerySet`. 

## Question 1: Identify users from queries (30 pts)

If you have a reasonable portrait of your users, then given an anonymous query you should be able to identify the user who issued that query with a reasonable accuracy. This is useful for "privacy-aware" personalisation where we still want to offer some personalisation even without any identifying information. We can effectively frame the problem of identifying users from queries as a *text classification* problem where the queries are the inputs and the users are the class labels. 

The Bing search log data set is completely de-identified, so we don't actually have access to the information about individual users. Nevertheless, we can think of each country as a "user" (or a homogenous group of individual users) and try to understand such a "user" better. In what follows, we will use the words "country" and "user" interchangeably. 

Your task for this question is to complete the `TextClassifier` class below which has three methods:

* **`_load_data`**: It loads data from a `.tsv` file at the given `path` and returns a `pd.DataFrame` that is in the same format as the `.tsv` file (i.e., has the same column titles and order). **In this assignment, we only consider queries with a `PopularityScore` at least `min_pop_score` as "queries of interest" and countries from which at least `min_num_qs` number of "queries of interest" were issued as "users of interest".** We will only try to identify "users of interest" from "queries of interest", so other queries or users should be filtered out in your `pd.DataFrame`.


* **`fit`**: It uses data from a `.tsv` file at the given `path` as training data to fit the classifier `self.clf`, subject to the same `min_pop_score` and `min_num_qs` constraints. **You may use any official `sklearn` classifiers there are or write your own classifiers,** as long as the model stored in `self.clf` is a `BaseEstimator` and passes the `is_classifier` check (don't need to worry about this if you use an official classifier). In addition, feel free to define any auxiliary attributes in `__init__` as you see fit. 


* **`get_test_X_y`**: It turns data from a `.tsv` file at the given `path` into test data in the form of `X` and `y` as what would be expected by the `score` method of a typical classifier, subject to the same `min_pop_score` and `min_num_qs` constraints. **The label vector `y` should only contain labels that are also in your training data and correspondingly, so should the data matrix `X`.** You are assured that any call to `get_test_X_y` is always preceded by a call to `fit`, so that you have access to the labels in your training data.

In [2]:
import pandas as pd
import numpy as np

from sklearn.neighbors import KNeighborsClassifier 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import preprocessing

class TextClassifier:
    def __init__(self):
        
        self.clf = KNeighborsClassifier() # your choice of classifier
        self.lab_enc = preprocessing.LabelEncoder()
        
        self.vec = TfidfVectorizer(tokenizer=str.split)
        
        # any auxiliary attributes
        # YOUR CODE HERE
#         raise NotImplementedError()
        
    def _load_data(self, path, min_pop_score=10, min_num_qs=300):
        """
        Loads data into a DataFrame with the required filtering
        """
        df = pd.read_csv(path, sep='\t')
        df = df[df['PopularityScore']>=min_pop_score]
        v = df.Country.value_counts()
        df = df[df.Country.isin(v.index[v.gt(min_num_qs-1)])]
        
        
        
        # YOUR CODE HERE
#         raise NotImplementedError()
        
        return df
    
    def fit(self, path, min_pop_score=10, min_num_qs=300):
        """
        Fits the classifier using data from 'path'
        """
        
        df = self._load_data(path, min_pop_score = min_pop_score, min_num_qs = min_num_qs)
        
#         doc_term_mat = np.array()
#         for query in df['Query']:
        doc_term_mat = (self.vec.fit_transform([" ".join(x) for x in df['Query']]))
        
#         query_enc = self.lab_enc.fit_transform(np.array(df['Query']).reshape(1, -1))

#         country_mat = (self.vec.transform([" ".join(x) for x in df['Country']]))
        
        count_enc = self.lab_enc.fit_transform(np.array(df['Country']).reshape(-1, 1))
        
        self.countries = list(df['Country'].unique())
        
        self.clf.fit(doc_term_mat, count_enc)
        
        # YOUR CODE HERE
#         raise NotImplementedError()
        
        return self.clf
    
    def get_test_X_y(self, path, min_pop_score=10, min_num_qs=300):
        """
        Generates test data from 'path'
        """
        X, y = None, None
        
        test_df = self._load_data(path, min_pop_score=min_pop_score, min_num_qs=min_num_qs)
        
        test_df = test_df[test_df['Country'].isin(self.countries)]
        
        X = self.vec.transform([" ".join(x) for x in test_df['Query']])
        
#         query_enc = self.lab_enc.fit_transform(np.array(df['Query']).reshape(1, -1))
        
        y = self.lab_enc.transform(np.array(test_df['Country']).reshape(-1, 1))
        
#         y = (self.vec.transform([" ".join(x) for x in df['Country']]))
    
            
        
        
        # YOUR CODE HERE
#         raise NotImplementedError()
        
        return X, y

In [3]:
import pandas as pd
path = "assets/BingCoronavirusQuerySet/QueriesByCountry_2020-04-01_2020-04-30.tsv" 

min_pop_score = 10
min_num_qs = 700
df = pd.read_csv(path, sep = '\t')
df = df[df['PopularityScore']>=min_pop_score]

v = df.Country.value_counts()
df = df[df.Country.isin(v.index[v.gt(min_num_qs-1)])]

df.head()

# for x in df['Query']:
#     print(x)

,Date,Query,IsImplicitIntent,Country,PopularityScore
45,2020-04-01,corona virus statistics,False,Australia,19
302,2020-04-01,worldometer coronavirus,False,United Kingdom,27
606,2020-04-01,currys pc world,True,United Kingdom,35
860,2020-04-01,worldometer coronavirus,False,United States,14
960,2020-04-01,coronavirus,False,United States,63


Your `_load_data` method will be graded as usual by comparison with the correct answer. **(10 pts)**

In [4]:
# Autograder tests for loading data
import math
import pandas as pd

# This won't change in the hidden test. 
path = "assets/BingCoronavirusQuerySet/QueriesByCountry_2020-04-01_2020-04-30.tsv" 

# These may vary. 
min_pop_score, min_num_qs = 10, 300

stu_text_clf = TextClassifier()
stu_df = stu_text_clf._load_data(path, min_pop_score, min_num_qs)

# Some sanity checks
assert isinstance(stu_df, pd.DataFrame), "Q1: Your function should return a pd.DataFrame. "
assert stu_df["PopularityScore"].min() >= min_pop_score, f"Q1: Some queries in your pd.DataFrame have a PopularityScore < min_pop_score = {min_pop_score}. "
assert len(stu_df) >= len(pd.unique(stu_df["Country"])) * min_num_qs, f"Q1: Some countries in your pd.DataFrame have less than min_num_qs = {min_num_qs} queries. "

# Some hidden tests

del stu_text_clf, stu_df, min_pop_score, min_num_qs, path

Your `fit` and `get_test_X_y` methods will be graded by comparison with a simple baseline text classifier. Specifically, we will take each month's data as the training data and evaluate your text classifier on the test data generated from the following month. An example is given in the visible tests below, where we fit your text classifier on the data from April 2020 and use it to generate test data for May 2020. The autograder will compute the Matthews correlation coefficient as a performance measure and determine whether your text classifier outperforms the baseline. **(20 pts)**

In [5]:
# Autograder tests for text classification
from sklearn.base import BaseEstimator, is_classifier
from sklearn.utils import check_X_y
from sklearn.metrics import matthews_corrcoef

# These won't change in the hidden tests 
min_pop_score, min_num_qs = 10, 300

# These may vary.
train_path = "assets/BingCoronavirusQuerySet/QueriesByCountry_2020-04-01_2020-04-30.tsv"
test_path = "assets/BingCoronavirusQuerySet/QueriesByCountry_2020-05-01_2020-05-31.tsv"

# Define a text classfier and fit it with training data
stu_text_clf = TextClassifier()
stu_text_clf.fit(train_path, min_pop_score, min_num_qs)

# Some sanity checks
assert hasattr(stu_text_clf, "clf"), "Q1: Your text classifier should have an attribute 'clf'. "
assert isinstance(stu_text_clf.clf, BaseEstimator), "Q1: Your clf should be a sklearn Estimator. "
assert is_classifier(stu_text_clf.clf), "Q1: Your clf should be a sklearn Classifier. "
assert hasattr(stu_text_clf.clf, "predict"), "Q1: Your clf should have a 'predict' method. "

# Generate test data
stu_X, stu_y = stu_text_clf.get_test_X_y(test_path, min_pop_score, min_num_qs)

# check that X and y are valid
check_X_y(stu_X, stu_y, accept_sparse=True)


# Some hidden tests

del stu_text_clf, stu_X, stu_y, min_pop_score, min_num_qs, train_path, test_path

In [6]:
# stu_X, stu_y = stu_text_clf.get_test_X_y(test_path, min_pop_score, min_num_qs)

In [7]:
from sklearn.base import BaseEstimator, is_classifier
from sklearn.utils import check_X_y
from sklearn.metrics import matthews_corrcoef

# These won't change in the hidden tests 
min_pop_score, min_num_qs = 10, 300

# These may vary.
train_path = "assets/BingCoronavirusQuerySet/QueriesByCountry_2020-04-01_2020-04-30.tsv"
test_path = "assets/BingCoronavirusQuerySet/QueriesByCountry_2020-05-01_2020-05-31.tsv"

# Define a text classfier and fit it with training data
stu_text_clf = TextClassifier()
stu_text_clf.fit(train_path, min_pop_score, min_num_qs)

# Some sanity checks
assert hasattr(stu_text_clf, "clf"), "Q1: Your text classifier should have an attribute 'clf'. "
assert isinstance(stu_text_clf.clf, BaseEstimator), "Q1: Your clf should be a sklearn Estimator. "
assert is_classifier(stu_text_clf.clf), "Q1: Your clf should be a sklearn Classifier. "
assert hasattr(stu_text_clf.clf, "predict"), "Q1: Your clf should have a 'predict' method. "

# Generate test data
stu_X, stu_y = stu_text_clf.get_test_X_y(test_path, min_pop_score, min_num_qs)

In [8]:
# check that X and y are valid
# stu_X.shape
stu_y.shape

(7310,)

## Question 2: Detect query drifts (10 pts)

In the previous question, we take a month's query logs as training data and use the query logs in the following month as test data to evaluate the performance of our text classifier (and in some sense how well we know about our users). This is under the assumption that queries issued by the same user are "similar" between two consecutive months. But to what extent is this assumption valid? Can there be a "query drift" over months? 

Let's try to detect possible query drifts using similarity measures. Complete the function below that computes the average cosine similarity between all pairs of queries issued by a user. Your function should first load the training data from `train_path` and the test data from `test_path` like you did in the previous question, subject to the same `min_pop_score` and `min_num_qs` constraints. Then it should fit a default `TfidfVectorizer` on the training data for converting queries into vectors. Next, **for countries present in both training and test data**, compute the average cosine similarity between all pairs of queries issued by each country. **Each pair consists of one query from the training data and the other from the test data.** If, for each country, you imagine the queries in the training and test data as forming two clusters respectively, then the similarity measure you are computing is also known as "average linkage". If the two clusters turn out to be "dissimilar", then there has probably been a query drift. 

Your function should output a `dict` that contains the above-mentioned average cosine similarity for each country, similar to:

```
{
    'Brazil': 0.1176541008844407,
    'United States': 0.0774657184345132,
    ...,
    'China': 0.15424835919750043
}
```

**Hint:** the vectors produced by a `TfidfVectorizer` are already normalised to unit length, which should greatly simplify the calculation of cosine similarity. 

In [9]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

def compute_avg_cos_sim(train_path, test_path, min_pop_score=10, min_num_qs=300):
    """
    Compute the average cosine similarity between all train-test pairs of queries for each country
    """
#     vec = TfidfVectorizer(tokenizer=str.split)
    vec = TfidfVectorizer()

    
    def load_data(path,min_pop_score = min_pop_score, min_num_qs = min_num_qs):
        df = pd.read_csv(path, sep='\t')
        df = df[df['PopularityScore']>=min_pop_score]
        v = df.Country.value_counts()
        df = df[df.Country.isin(v.index[v.gt(min_num_qs-1)])]
        
        return df
    
    train_df = load_data(train_path,min_pop_score = min_pop_score, min_num_qs = min_num_qs)
    
    countries = list(train_df['Country'].unique())
#     print(countries)
    
    test_df = load_data(test_path,min_pop_score = min_pop_score, min_num_qs = min_num_qs)
    
    test_df = test_df[test_df['Country'].isin(countries)]
    
    test_countries = list(test_df['Country'].unique())
#     print(test_countries)
    
#     print(test_df[test_df['Country']=='Other Country'].head())
    
#     print(test_df[test_df['Country']=='Other Country'])
    
    vec.fit(train_df['Query'])
    
#     doc_term_mat = (vec.fit_transform([" ".join(x) for x in train_df['Query']]))
    
    
    avg_cos_sim = {}
    
#     train_df['emb'] = vec.transform([" ".join(x) for x in train_df['Query']])
#     test_df['emb'] = vec.transform([" ".join(x) for x in test_df['Query']])
    

    
    for country in test_countries:
#         cos_sim = []
        
        train2 = train_df.copy()
        test2 = test_df.copy()
        
        train2 = train2[train2['Country']==country]
        test2 = test2[test2['Country']==country]
        
        
        train_doc_term_mat = (vec.transform(train2['Query']))
        
        test_doc_term_mat = (vec.transform(test2['Query']))
        
#         if len(train2) <= len(test2):
#             for idx, query in enumerate(train_doc_term_mat):
#                 cos_sim.append(cosine_similarity(train_doc_term_mat[idx],test_doc_term_mat[idx]))
#         else:
#             for idx, query in enumerate(test_doc_term_mat):
#                 cos_sim.append(cosine_similarity(train_doc_term_mat[idx],test_doc_term_mat[idx]))

        cos_sim = cosine_similarity(train_doc_term_mat,test_doc_term_mat)    
        avg_cos_sim[country] = np.mean(cos_sim)
        
    
    return avg_cos_sim

In [10]:
# Autograder tests
import math

# These won't change in the hidden tests 
min_pop_score, min_num_qs = 10, 300

# These may vary.
train_path = "assets/BingCoronavirusQuerySet/QueriesByCountry_2020-04-01_2020-04-30.tsv"
test_path = "assets/BingCoronavirusQuerySet/QueriesByCountry_2020-05-01_2020-05-31.tsv"

stu_avg_cos_sim = compute_avg_cos_sim(train_path, test_path, min_pop_score, min_num_qs)

# Some sanity checks
assert isinstance(stu_avg_cos_sim, dict), "Q2: Your function should output a dictionary. "
assert all([isinstance(cos_sim, float) for cos_sim in stu_avg_cos_sim.values()]), "Q2: All values of your dictionary should be Python floats. "
assert all([0 <= cos_sim <= 1 for cos_sim in stu_avg_cos_sim.values()]), "Q2: All cosine similarity must be between 0 and 1. "

# Some hidden tests

del stu_avg_cos_sim, min_pop_score, min_num_qs, train_path, test_path

In [11]:
# Autograder tests
import math

# These won't change in the hidden tests 
min_pop_score, min_num_qs = 10, 300

# These may vary.
train_path = "assets/BingCoronavirusQuerySet/QueriesByCountry_2020-04-01_2020-04-30.tsv"
test_path = "assets/BingCoronavirusQuerySet/QueriesByCountry_2020-05-01_2020-05-31.tsv"

stu_avg_cos_sim = compute_avg_cos_sim(train_path, test_path, min_pop_score, min_num_qs)



In [12]:
stu_avg_cos_sim

{'Mexico': 0.14687666261776725,
 'Italy': 0.062444150418798666,
 'United States': 0.07746571843451316,
 'South Africa': 0.14587177470675886,
 'United Kingdom': 0.03749019820714714,
 'India': 0.14645447497468403,
 'Germany': 0.11905917132830261,
 'Australia': 0.09791953156721327,
 'Canada': 0.12120900130563451,
 'China': 0.1542483591975002,
 'Brazil': 0.11765410088444075,
 'Austria': 0.12473110582384699}

## Question 3: Group users by queries (10 pts)

So far our analysis has been focused on queries issued by the same user. A user-centric analysis is also possible; for example, we could try to identify groups of users who have issued "similar" queries in a given month. Those users in the same group could share similar information needs. 

Complete the function below for clustering users into such groups. Like before, your function should first load a `.tsv` file from the given `path` subject to the same `min_pop_score` and `min_num_qs` constraints. Use a default `TfidfVectorizer` for converting queries into vectors. We will represent each user as the average vector of all the queries issued by that user in the given month. Then apply a default `KMeans` with `n_clusters=num_clusters` and `random_state=42` to the **user vectors**. Finally, your function should output a `dict` showing the cluster membership, similar to:

```
{
    'Cluster 1': ['India', 'Australia', ..., 'China'],
    'Cluster 0': ['United Kingdom', 'United States', ..., 'France'],
    'Cluster 2': ['Mexico', 'Argentina']
}
```

The cluster labels (e.g., `'Cluster 1'`) and the countries within each cluster can be in any order. 

⚠️ Your input to `KMeans` would be a $N\times D$ matrix where each row stores the vector representation for each user. **Please make sure the rows are ordered in the same way as how the countries are ordered in the `.tsv` file.** In other words, please do not (inadvertently) sort the countries in any way. Use `sort=False` option if you want to use `df.groupby`. You may find `pd.unique` useful. This is because `KMeans` is sensitive to the input order. ⚠️

In [20]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans

def cluster_users(path, num_clusters, min_pop_score=10, min_num_qs=300):
    """
    Cluster users into groups based on queries
    """
    
    vec = TfidfVectorizer()
    clf = KMeans(n_clusters = num_clusters, random_state = 42)
    
    
    def load_data(path,min_pop_score = min_pop_score, min_num_qs = min_num_qs):
        df = pd.read_csv(path, sep='\t')
        df = df[df['PopularityScore']>=min_pop_score]
        v = df.Country.value_counts()
        df = df[df.Country.isin(v.index[v.gt(min_num_qs-1)])]
        
        return df
    
    df = load_data(path)
#     df['month'] = pd.DatetimeIndex(df['Date']).month
    
#     print(df.head())
    vec.fit(df['Query'])
    
    countries = df['Country'].unique()
    print(countries)
    
    X = []
    for country in countries:
        df2 = df.copy()
        
        df2 = df2[df2['Country']==country]
        
        train_doc_term_mat = (vec.transform(df2['Query']))
        
        avg_vec = np.mean(train_doc_term_mat, axis = 0)
        
        X.append(avg_vec)
        
#     print(X)

    user_clusters = None
    
    X = np.vstack(X)
    
    kmeans = clf.fit_predict(np.array(X))
    
#     print(np.array(X).reshape(-1,1))
    
    clus_list = list(zip(countries,kmeans))
    
    clusters = pd.DataFrame(clus_list, columns = ['Country','Cluster'])
    
#     print(clusters)
        
    
    user_clusters = {'Cluster '+ str(k): list(v) for k, v in clusters.groupby('Cluster',sort=False)['Country']}
        
        
    
    
    # YOUR CODE HERE
#     raise NotImplementedError()
    
    return user_clusters

In [21]:
# Autograder tests

# These won't change in the hidden tests 
min_pop_score, min_num_qs = 10, 300

# These may vary.
path = "assets/BingCoronavirusQuerySet/QueriesByCountry_2020-04-01_2020-04-30.tsv"
num_clusters = 3

stu_user_clusters = cluster_users(path, num_clusters, min_pop_score, min_num_qs)

# Some sanity checks
assert isinstance(stu_user_clusters, dict), "Q3: Your function should output a dictionary. "
assert len(stu_user_clusters) == num_clusters, "Q3: The length of your dictionary should be the same as num_clusters. "

stu_countries = [cty for v in stu_user_clusters.values() for cty in v]
assert len(set(stu_countries)) == len(stu_countries), "Q3: Some countries belong to more than one cluster. "

# Some hidden tests

del stu_user_clusters, min_pop_score, min_num_qs, stu_countries, path

['India' 'Australia' 'Indonesia' 'United Kingdom' 'United States'
 'Germany' 'Canada' 'Brazil' 'Puerto Rico' 'Austria' 'Italy' 'France'
 'Mexico' 'South Africa' 'Argentina' 'Malaysia' 'China']


In [22]:
# Autograder tests

# These won't change in the hidden tests 
min_pop_score, min_num_qs = 10, 300

# These may vary.
path = "assets/BingCoronavirusQuerySet/QueriesByCountry_2020-04-01_2020-04-30.tsv"
num_clusters = 3

stu_user_clusters = cluster_users(path, num_clusters, min_pop_score, min_num_qs)


['India' 'Australia' 'Indonesia' 'United Kingdom' 'United States'
 'Germany' 'Canada' 'Brazil' 'Puerto Rico' 'Austria' 'Italy' 'France'
 'Mexico' 'South Africa' 'Argentina' 'Malaysia' 'China']


In [23]:
stu_user_clusters

{'Cluster 1': ['India',
  'Australia',
  'Indonesia',
  'Canada',
  'Puerto Rico',
  'South Africa',
  'Malaysia',
  'China'],
 'Cluster 0': ['United Kingdom',
  'United States',
  'Germany',
  'Brazil',
  'Austria',
  'Italy',
  'France'],
 'Cluster 2': ['Mexico', 'Argentina']}